In [ ]:
#Import all the necessary libraries
from tkinter import *
import tkinter
import pyaudio
import wave
import matplotlib.pyplot as plt
import numpy as np
import keras
import librosa
from PIL import Image,ImageTk



#Define the tkinter instance
win= Tk()
win.title("Speech Emotion Recognition")

#Define the size of the tkinter frame
win.geometry("700x300")

#Define the working of the button

def record():
    FRAMES_PER_BUFFER = 3200
    FORMAT = pyaudio.paInt16
    CHANNELS = 1
    RATE = 16000

    pa = pyaudio.PyAudio()

    stream = pa.open(
        format=FORMAT,
        channels=CHANNELS,
        rate=RATE,
        input=True,
        frames_per_buffer=FRAMES_PER_BUFFER
    )

    print('start recording')

    seconds = 8
    frames = []
    second_tracking = 0
    second_count = 0
    for i in range(0, int(RATE/FRAMES_PER_BUFFER*seconds)):
        data = stream.read(FRAMES_PER_BUFFER)
        frames.append(data)
        second_tracking += 1
        if second_tracking == RATE/FRAMES_PER_BUFFER:
            second_count += 1
            second_tracking = 0
            print(f'Time Left: {seconds - second_count} seconds')


    stream.stop_stream()
    stream.close()
    pa.terminate()

    obj = wave.open('liveaudio.wav', 'wb')
    obj.setnchannels(CHANNELS)
    obj.setsampwidth(pa.get_sample_size(FORMAT))
    obj.setframerate(RATE)
    obj.writeframes(b''.join(frames))
    obj.close()


    file = wave.open('liveaudio.wav', 'rb')

    sample_freq = file.getframerate()
    frames = file.getnframes()
    signal_wave = file.readframes(-1)

    file.close()

    time = frames / sample_freq


    # if one channel use int16, if 2 use int32
    audio_array = np.frombuffer(signal_wave, dtype=np.int16)

    times = np.linspace(0, time, num=frames)

    plt.figure(figsize=(15, 5))
    plt.plot(times, audio_array)
    plt.ylabel('Signal Wave')
    plt.xlabel('Time (s)')
    plt.xlim(0, time)
    plt.title('The Thing I Just Recorded!!')
    plt.show()
label=''
def predict():
    class livePredictions:

        def __init__(self, path, file):
            self.path = path
            self.file = file

        def load_model(self):
            self.loaded_model = keras.models.load_model(self.path)
            return self.loaded_model.summary()

        def makepredictions(self):
            data, sampling_rate = librosa.load(self.file)
            mfccs = np.mean(librosa.feature.mfcc(y=data, sr=sampling_rate, n_mfcc=40).T, axis=0)
            x = np.expand_dims(mfccs, axis=1)
            x = np.expand_dims(x, axis=0)
            #predictions = self.loaded_model.predict_classes(x)

            predictions=self.loaded_model.predict(x) 
            predictions=np.argmax(predictions,axis=1)

            print("Prediction is", " ", self.convertclasstoemotion(predictions))
            
        @staticmethod
        def convertclasstoemotion(pred):
            label_conversion = {'0': 'neutral',
                                '1': 'calm',
                                '2': 'happy',
                                '3': 'sad',
                                '4': 'angry',
                                '5': 'fearful',
                                '6': 'disguist',
                                '7': 'surprised'}

            for key, value in label_conversion.items():
                if int(key) == pred:
                    label = value
            
            i1=Image.open('/Users/srinivasreddy/Documents/neutral.jpeg')
            n1=i1.resize((180,180))
            n1.save('neutral.jpeg')
            
            i2=Image.open('/Users/srinivasreddy/Documents/calm.jpeg')
            n2=i2.resize((180,180))
            n2.save('calm.jpeg')
            
            i3=Image.open('/Users/srinivasreddy/Documents/happy.jpeg')
            n3=i3.resize((180,180))
            n3.save('happy.jpeg')
            
            i4=Image.open('/Users/srinivasreddy/Documents/sad.jpeg')
            n4=i4.resize((170,170))
            n4.save('sad.jpeg')
            
            i5=Image.open('/Users/srinivasreddy/Documents/angry.png')
            n5=i5.resize((170,170))
            n5.save('angry.png')
            
            i6=Image.open('/Users/srinivasreddy/Documents/fearful.jpeg')
            n6=i6.resize((180,180))
            n6.save('fearful.jpeg')
            
            i7=Image.open('/Users/srinivasreddy/Documents/disguist.jpeg')
            n7=i7.resize((170,170))
            n7.save('disguist.jpeg')
            
            i8=Image.open('/Users/srinivasreddy/Documents/surprise.jpeg')
            n8=i8.resize((170,170))
            n8.save('surprise.jpeg')
            
            global ej_image
            if label=='neutral':
                ej_image=ImageTk.PhotoImage(Image.open('neutral.jpeg'))
            elif label=='calm':
                ej_image=ImageTk.PhotoImage(Image.open('calm.jpeg'))
            elif label=='happy':
                ej_image=ImageTk.PhotoImage(Image.open('happy.jpeg'))
            elif label=='sad':
                ej_image=ImageTk.PhotoImage(Image.open("sad.jpeg"))
            elif label=='angry':
                ej_image=ImageTk.PhotoImage(Image.open('angry.png'))
            elif label=='fearful':
                ej_image=ImageTk.PhotoImage(Image.open('fearful.jpeg'))
            elif label=='disguist':
                ej_image=ImageTk.PhotoImage(Image.open('disguist.jpeg'))
            else:
                ej_image=ImageTk.PhotoImage(Image.open('surprise.jpeg'))
            text.image_create(END,image=ej_image)
            print(label)
            return label
        
    pred = livePredictions(path='/Users/srinivasreddy/testing10_model.h5',file='/Users/srinivasreddy/Downloads/TESS Toronto emotional speech set data/OAF_happy/OAF_bean_happy.wav')
    pred.load_model()
    pred.makepredictions()
    
    
click_btn= PhotoImage(file='/Users/srinivasreddy/Documents/record.png')

img_label= Label(image=click_btn)

button= Button(win,command= record,borderwidth=10)
button.pack(side=LEFT)

btn1=Button(win, text='predict',command=predict,borderwidth=5,height= 5, width=10)
btn1.place(x=300,y=100)

ej_image=PhotoImage(file='/Users/srinivasreddy/Documents/record.png')
#text.image_create(END,image=ej_image)

text=Text(win,borderwidth=10)
#text.pack(side=RIGHT)
text.place(x=490, y= 47, width= 200, height= 200)

win.mainloop()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_3 (Conv1D)           (None, 40, 64)            384       
                                                                 
 activation_4 (Activation)   (None, 40, 64)            0         
                                                                 
 dropout_3 (Dropout)         (None, 40, 64)            0         
                                                                 
 max_pooling1d_2 (MaxPooling  (None, 10, 64)           0         
 1D)                                                             
                                                                 
 conv1d_4 (Conv1D)           (None, 10, 128)           41088     
                                                                 
 activation_5 (Activation)   (None, 10, 128)           0         
                                                      